In [1]:
from agents import Agent, Runner, trace
from agents.tool import FunctionTool
from dotenv import load_dotenv
import requests, os


In [2]:
load_dotenv(override=True)

True

In [3]:
perplexity_api_key = os.getenv('PERPLEXITY_API_KEY')

if perplexity_api_key:
    print(f"Perplexity API Key exists and begins {perplexity_api_key[:4]}")
else:
    print("Perplexity API Key not set (and this is optional)")

model_name="gpt-5-nano"
perp_model_name="sonar"

Perplexity API Key exists and begins pplx


In [4]:
def perplexity_search(query: str) -> str:
    response = requests.post(
        url="https://api.perplexity.ai/chat/completions",
        headers={
            "Authorization": f"Bearer {perplexity_api_key}",
            "Content-Type": "application/json"
        },
        json={
            "model": model_name,
            "messages": [{"role": "user", "content": query}],
            "temperature": 0
        }
    )
    response.raise_for_status()
    return response.json().choices[0].message.content

perplexity_params_schema = {
    "type": "object",
    "properties": {
        "query": {
            "type": "string",
            "description": "The search query to look up on the web"
        }
    },
    "required": ["query"],
    "additionalProperties": False
}

def on_invoke_perplexity_tool(ctx, args):
    query = args["query"]
    return perplexity_search(query)

perplexity_tool = FunctionTool(
    name="perplexity_search",
    description="Search the web using Perplexity for up-to-date factual information",
    params_json_schema=perplexity_params_schema,
    on_invoke_tool=on_invoke_perplexity_tool,
)



In [5]:
instrs = 'You are a English dictionary. Everytime user asks you should give ten english words which are medium to high complexity, provide its meaning, provide atleast one synonym and atleast one antonym of each word. The idea is to grow vocabulary of the user. The users can be from 10 years old and above. Give new set of words and repeat words only after atleast 10 days'



In [6]:
agent = Agent(name='Word Dictionary', instructions=instrs, tools=[perplexity_tool], model = model_name)

In [7]:
result = await Runner.run(agent, "Give me next set of words")
print(result)

RunResult:
- Last agent: Agent(name="Word Dictionary", ...)
- Final output (str):
    Here are ten medium-high complexity words to grow your vocabulary:
    
    1) Aberration — noun: a departure from what is normal or expected; synonyms: deviation, anomaly; antonyms: normality, regularity.
    2) Cogent — adjective: clear, logical, and convincing; synonyms: persuasive, compelling; antonyms: weak, unconvincing.
    3) Dearth — noun: a scarcity or lack of something; synonyms: paucity, shortage; antonyms: abundance, plenty.
    4) Eloquent — adjective: fluent or persuasive in speaking or writing; synonyms: articulate, expressive; antonyms: inarticulate, tongue-tied.
    5) Fastidious — adjective: very attentive to detail; hard to please; synonyms: meticulous, picky; antonyms: indifferent, careless.
    6) Gregarious — adjective: fond of company; sociable; synonyms: sociable, outgoing; antonyms: solitary, introverted.
    7) Ostentatious — adjective: showy, designed to attract notice; syn